In [1]:
import numpy as np
import yaml
import emcee
import matplotlib.pyplot as plt
import corner

In [2]:
configfile = 'config.yaml'

In [3]:
import torch
import torch.nn as nn

In [4]:
class TwoHiddenLayerNN(nn.Module):
    def __init__(self, input_dim, output_dim, nodes):
        super(TwoHiddenLayerNN, self).__init__()
        self.model = nn.Sequential(nn.Linear(input_dim, 1024), 
                                   nn.ReLU(),
                                   nn.Linear(1024, 1024), 
                                   nn.ReLU(),
                                   nn.Linear(1024, output_dim), 
        )

    def forward(self, x):    
        return self.model(x)

In [5]:
em_model = TwoHiddenLayerNN(7, 400, 0)
em_model.load_state_dict(torch.load("../../cocoa/Cocoa/trainedmodel"))
em_model.eval()

TwoHiddenLayerNN(
  (model): Sequential(
    (0): Linear(in_features=7, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1024, out_features=400, bias=True)
  )
)

In [6]:
norms = np.load(f"/sps/lsst/users/bbiswas/ICTS-School/norms.npy", allow_pickle=True).item()

In [7]:
def hard_prior(theta, params_prior):
    is_lower_than_min = bool(np.sum(theta < params_prior[:,0]))
    is_higher_than_max = bool(np.sum(theta > params_prior[:,1]))
    if is_lower_than_min or is_higher_than_max:
        return -np.inf
    else:
        return 0.
    
def gaussian_prior(theta, params_prior):
    mu  = params_prior[:,0]
    std = params_prior[:,1]
    y = (theta - mu) / std
    return -0.5 * np.sum(y * y)

def ln_prior(theta):
    prior_flat = 0.
    prior_gauss = 0.
    
    if(len(config.flat_prior_indices) != 0):
        flat_prior_theta     = theta[config.flat_prior_indices]
        prior_flat    += hard_prior(flat_prior_theta, config.flat_prior_parameters)
    if(len(config.gaussian_prior_indices) != 0):
        gaussian_prior_theta = theta[config.gaussian_prior_indices]
        prior_gauss   += gaussian_prior(gaussian_prior_theta, config.gaussian_prior_parameters)
        
    return prior_flat + prior_gauss

def emulator_predict(theta):
    x_norm = (theta-norms['x_mean'])/norms['x_std']
    y_predict = em_model(torch.tensor(x_norm, dtype=torch.float32))
    y_unnorm = y_predict.detach().numpy()*norms['y_std'] + norms['y_mean']
    return y_unnorm[config.mask_bool[:400]]

def ln_lkl(theta):
    y_predict = emulator_predict(theta)
    # print(y_predict)
    diff = config.dv-y_predict
    # print(diff)
    log_lk = - diff.T @ config.inv_cov @ diff
    return log_lk
    

def ln_posterior(theta):
    print("lkl " + str(ln_lkl(theta)))
    print("prior " + str(ln_prior(theta)))
    return ln_lkl(theta) + ln_prior(theta)

In [8]:
norms['x_mean']

array([2.35153000e+00, 9.68236253e-01, 7.14129226e+01, 5.07577486e-02,
       2.52832324e-01, 1.90734405e-01, 4.43486925e-01])

In [9]:
class Config:
    def __init__(self, configfile):
        with open(configfile, "r") as stream:
            config_args = yaml.safe_load(stream)
        self.priors = config_args['priors']
        self.data   = config_args['data']
        self.get_prior_parameters()
        self.get_data()
    
    def get_prior_parameters(self):
        self.gaussian_prior_indices = []
        self.gaussian_prior_parameters = []

        self.flat_prior_indices    = []
        self.flat_prior_parameters = []

        ind = 0
        self.ndim = len(self.priors)
        
        for x in self.priors:
            prior = self.priors[x]
            dist  = prior['dist']
            if(dist=='gauss'):
                self.gaussian_prior_indices.append(ind)
                self.gaussian_prior_parameters.append([prior['loc'], prior['scale']])
            else:
                self.flat_prior_indices.append(ind)
                self.flat_prior_parameters.append([prior['min'], prior['max']])
            ind += 1
        
        self.gaussian_prior_parameters = np.array(self.gaussian_prior_parameters)
        self.flat_prior_parameters = np.array(self.flat_prior_parameters)

    def get_data(self):
        #read in DES measurement
        self.mask = np.genfromtxt(self.data['maskfile'])[:,1]
        self.mask_bool = self.mask.astype(bool)
        
        d = np.genfromtxt(self.data['dvfile'])[:,1]
        ndata = d.shape[0]
        
        cov =np.zeros((ndata,ndata))
        data = np.genfromtxt(self.data['covfile'])
        for i in range(0,data.shape[0]):
            cov[int(data[i,0]),int(data[i,1])] = data[i,2]
            cov[int(data[i,1]),int(data[i,0])] = data[i,2]
            
        self.cov = cov[self.mask_bool][:,self.mask_bool]
        self.dv  = d[self.mask_bool]
        
        self.inv_cov = np.linalg.inv(self.cov)

In [10]:
config = Config(configfile)

In [11]:
# Starting point for the emcee walkers with mean theta0 and rms theta_std

theta0 = 0.5 * (config.flat_prior_parameters[:,1] + config.flat_prior_parameters[:,0])
theta_std = 0.1 * (config.flat_prior_parameters[:,1]-config.flat_prior_parameters[:,0])

In [12]:
(ln_lkl(theta0))

-20739.266666712396

In [13]:
np.set_printoptions(suppress = True)

In [14]:
pos0

NameError: name 'pos0' is not defined

In [15]:
ndim, nwalkers = config.ndim, 20
pos0 = theta0 + theta_std * np.random.randn(nwalkers, ndim)

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, ln_posterior)
sampler.run_mcmc(pos0, 5000);

lkl -36668.881199231604
prior 0.0
lkl -1898.8025371695146
prior 0.0
lkl -25680.604929754794
prior 0.0
lkl -18134.264740648938
prior 0.0
lkl -1485.3901607365533
prior 0.0
lkl -26372.759393884317
prior 0.0
lkl -35418.131235960675
prior 0.0
lkl -50820.25834203996
prior 0.0
lkl -12383.536335778419
prior 0.0
lkl -7273.965297810626
prior 0.0
lkl -47349.08531033744
prior 0.0
lkl -41788.66762460327
prior 0.0
lkl -56054.228530277134
prior 0.0
lkl -18734.39165161374
prior 0.0
lkl -34420.74721900766
prior 0.0
lkl -41920.18970669324
prior 0.0
lkl -5755.747572720026
prior 0.0
lkl -15188.378906631702
prior 0.0
lkl -14944.109667501758
prior 0.0
lkl -30158.812524471516
prior 0.0
lkl -23569.372921272017
prior 0.0
lkl -8967.662243585024
prior 0.0
lkl -12085.65075349924
prior 0.0
lkl -58480.107668293764
prior 0.0
lkl -32316.999593116303
prior 0.0
lkl -51982.186149029505
prior 0.0
lkl -10899.129284080922
prior 0.0
lkl -42604.2200854232
prior 0.0
lkl -39964.96254170178
prior 0.0
lkl -29465.13472178791
prio

In [ ]:
fig, ax = plt.subplots(7,1,figsize=(10,17.5))

for i in range(7):
    ax[i].plot(sampler.chain[:,:,i].T, 'k-', lw=0.3)

plt.show()    

In [ ]:
samples = sampler.chain[:,1000:].reshape((-1,7))

In [ ]:
corner.corner(samples)
plt.show()